In [ ]:
import cv2
import os
import numpy as np
from PIL import Image
import skimage
from skimage.metrics import structural_similarity
from skimage import data, draw, io
import matplotlib.pyplot as plt

In [ ]:
#reads in video to init_video
with open("dataset/Videos/test2.rgb", "rb") as f:
    v = np.fromfile(f, np.dtype('B'))
#setting the number of pixels in frame
pxct = 129600
#setting the number of frames
frct = (int)(v.size/(3*pxct))
init_video = []
for frame_number in range(frct):
        start = (frame_number) * pxct * 3
        curr_frame = v[start:start + 3 * pxct]
        final_frame = np.reshape(curr_frame, (480,270,3), order = 'F')
        final_frame = np.rot90(final_frame,3)
        final_frame = np.flip(final_frame,1)
        
        #searching these final frames for logos (new cell)
        init_video.append(final_frame)

In [ ]:
#generate cuts for advertisement
advert_frames = []
for frame_number in range(frct - 1):
    #calculate new shots with histogram
    #less means more similar
    hist1 = cv2.calcHist([init_video[frame_number]],[0],None,[256],[0,256])
    hist2 = cv2.calcHist([init_video[frame_number + 1]],[0],None,[256],[0,256])
    score=cv2.compareHist(hist1,hist2,cv2.HISTCMP_BHATTACHARYYA)
    #store score for ssi
    #less means more similar
    #i1 = cv2.cvtColor(init_video[frame_number], cv2.COLOR_BGR2GRAY)
    #i2 = cv2.cvtColor(init_video[frame_number + 1], cv2.COLOR_BGR2GRAY)
    #score2 = structural_similarity(i1, i2)
    #score2 = 1-score2 
    #frame_stats[frame_number][1] = score2
    
    #score indicates sensitivity for shot changes
    if score > 0.2:
        advert_frames.append(frame_number)
        advert_frames.append(frame_number+1)
        #saves the image, currently used for debugging
        #if score + score2 > 0.3:
        data = Image.fromarray(init_video[frame_number]).convert('RGB')
        data.save('images_cut/{}.png'.format(frame_number))
        data = Image.fromarray(init_video[frame_number+1]).convert('RGB')
        data.save('images_cut/{}.png'.format(frame_number+1))
#save beginning and end of cuts in number of frames
#storing the advertisements to replace with
#indexed the same way as that of cuts_begin and cuts_end
#if not to be replaced, a blank line
replaced_advertisements = []
cuts_begin = []
cuts_end = []
for i in range(len(advert_frames)):
    for j in range(i,len(advert_frames)):
        if advert_frames[j] - advert_frames[i] == 449:
            cuts_begin.append(advert_frames[i])
            cuts_end.append(advert_frames[j])

In [ ]:
cuts_begin

In [ ]:
#cuts_begin has all starts of clips of 15seconds (449 frames) long exactly
#cuts_end has all the ends
print(advert_frames)
cuts_begin = []
cuts_end = []
for i in range(len(advert_frames)):
    for j in range(i,len(advert_frames)):
        if advert_frames[j] - advert_frames[i] == 449:
            cuts_begin.append(advert_frames[i])
            cuts_end.append(advert_frames[j])
print(cuts_begin)            

In [ ]:
#reading in all other advertisements
#saves in a dictionary with first part assumed to be the company name

advertisements = {}
PATHS_TO_ADVERTISEMENTS = "dataset/Ads"
for file in os.listdir(PATHS_TO_ADVERTISEMENTS):
    filename = os.fsdecode(file)
    if filename.endswith(".rgb"): 
        vpath = os.path.join(PATHS_TO_ADVERTISEMENTS, filename)
        with open(vpath, "rb") as f:
            v = np.fromfile(f, np.dtype('B'))
        #setting the number of pixels in frame
        pxct = 129600
        #setting the number of frames
        frct = (int)(v.size/(3*pxct))
        video = []
        for frame_number in range(frct):
            start = (frame_number) * pxct * 3
            curr_frame = v[start:start + 3 * pxct]
            final_frame = np.reshape(curr_frame, ( 480,270, 3), order = 'F')
            final_frame = np.rot90(final_frame,3)
            final_frame = np.flip(final_frame,1)
            video.append(final_frame)
        #Get the name of the company
        key = filename.split('_')[0]
        advertisements[key] = video


    

In [ ]:
#splicing in videos with the names 
#save beginning and end of cuts in number of frames
cuts_begin = [2585,7006]
cuts_end = [3034, 7455]
#storing the advertisements to replace with
#indexed the same way as that of cuts_begin and cuts_end
#if not to be replaced, an empty string
replaced_advertisements = ["Starbucks", "Subway"]
final_video = []
for i in range (len(cuts_begin)):
    if i == 0:
        final_video = init_video[:cuts_begin[i]] + advertisements[replaced_advertisements[i]] + init_video[cuts_end[i]:]
    else:
        #further work required to sync frames iff replacement ad lengths are variable
        final_video = final_video[:cuts_begin[i]] + advertisements[replaced_advertisements[i]] + final_video[cuts_end[i]:]
        

In [ ]:
t = init_video[1388]
data = Image.fromarray(t).convert('RGB')
data.save('box-test.png'.format(1))
img = cv2.imread('box-test.png')
y = np.flip(t,axis = 2)

In [ ]:
# 0th index of returned list is boolean
# 1st index is stuff needed to draw
def full_logo_recognition(frame, logo):
    #read in logo 
    graylogo = cv2.cvtColor(logo, cv2.COLOR_BGR2GRAY)

# #change to grayscale
    grayframe = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    #sift = cv2.SIFT_create()
    #logopoints, logodescriptors = sift.detectAndCompute(graylogo, None)
    #framepoints, framedescriptors = sift.detectAndCompute(grayframe, None)
    #print(logopoints)
    #bf = cv2.BFMatcher(cv2.NORM_L1, crossCheck=True)
    #bf = cv2.BFMatcher()

    sift = cv2.ORB_create(nfeatures=1500,edgeThreshold = 16, patchSize=16)
    logopoints, logodescriptors = sift.detectAndCompute(graylogo, None)
    framepoints, framedescriptors = sift.detectAndCompute(grayframe, None)
    
    bf = cv2.BFMatcher(cv2.NORM_HAMMING)
    
    matches = bf.knnMatch(logodescriptors,framedescriptors, k=2)

    # Apply ratio test
    good_points = []
    good_points_nolist = []
    for m,n in matches:
        if m.distance < 0.8*n.distance:
            good_points.append([m])
            good_points_nolist.append(m)
    #print(good_points)

#cv2.drawMatchesKnn expects list of lists as matches.
    img3 = cv2.drawMatchesKnn(logo, logopoints, frame, framepoints, good_points, None,
                              matchColor=(0, 255, 0), matchesMask=None,
                              singlePointColor=(255, 0, 0), flags=0)
    returnlist = [good_points_nolist, logopoints, framepoints]
    return ([len(good_points) >= 0.08*len(logodescriptors), returnlist])

In [ ]:
def draw_frames(frame, good_points_nolist, logopoints, framepoints, logodims, logoresize, color):
    list_kp1 = []
    list_framepoints = []
    for mat in good_points_nolist:

    # Get the matching keypoints for each of the images
        img1_idx = mat.queryIdx
        img2_idx = mat.trainIdx

    # x - columns
    # y - rows
    # Get the coordinates
        (x1, y1) = logopoints[img1_idx].pt
        (x2, y2) = framepoints[img2_idx].pt

    # Append to each list
        #list_kp1.append((x1, y1))
        list_framepoints.append((x2, y2))
        img = np.ascontiguousarray(frame, dtype=np.uint8)
        #get new saved png frame

    if(len(list_framepoints) > 0):
        average = len(list_framepoints);
        x=0
        y=0
        x_tempmin=0
        y_tempmin=0
        for pt in list_framepoints: 
            x_tempmin = min([x_tempmin,pt[0]])
            y_tempmin = min([y_tempmin,pt[1]])
            x = x+pt[0]
            y = y+pt[1] 
        x_average = int(x/average)
        y_average = int(y/average)

        center_coordinates = (x_average, y_average)
        radius = 5
        
        thickness = 4
        #image = cv2.circle(img, center_coordinates, radius, color, thickness)
        logoresizex = ((x_tempmin - x_average)/logodims[0] + 0.3)/2
        logoresizey = ((y_tempmin - y_average)/logodims[1] + 0.3)/2
        #logoresizey = logoresize
        #logoresizex = logoresize
        xlsz = int(logodims[0] * logoresizex)
        ylsz = int(logodims[1] * logoresizey) 
        xmin = max([x_average - xlsz,1])
        ymin = max([y_average - ylsz,1])
        xmax = min([x_average + xlsz,479])
        ymax = min([y_average + ylsz,269])
        image = cv2.rectangle(img, (int(xmin),int(ymin)), (int(xmax),int(ymax)), color, thickness)
        return np.flip(image, axis = 2)

In [ ]:
#final
scale = cv2.imread('Dataset/Brand Images/{company}_logo.bmp'.format(company = "starbucks"))
scale_percent = 30 # percent of original size
width = int(scale.shape[1] * scale_percent / 100)
height = int(scale.shape[0] * scale_percent / 100)
dim = (width, height)
resized = cv2.resize(scale, dim, interpolation = cv2.INTER_AREA)
st = resized
sw = cv2.imread('Dataset/Brand Images/{company}_logo.bmp'.format(company = "subway"))
for k in range(0,1860):
    try:
        frame = init_video[k]
        stretarr = full_logo_recognition(frame, st)
        swretarr = full_logo_recognition(frame, sw)
        if stretarr[0]:
            init_video[k] = draw_frames(frame,stretarr[1][0],stretarr[1][1],stretarr[1][2], [480,130], 0.3, (0, 255, 0))
        if swretarr[0]:
            init_video[k] = draw_frames(frame,swretarr[1][0],swretarr[1][1],swretarr[1][2], [480,130], 0.3, ( 255,0, 0))
    except:
        print (k)
for k in range(1860,2550):
    try:
        frame = init_video[k]
        stretarr = full_logo_recognition(frame, st)
        if stretarr[0]:
            init_video[k] = draw_frames(frame,stretarr[1][0],stretarr[1][1],stretarr[1][2], [480,130], 0.3, ( 255,0, 0))
    except:
        print (k)
for k in range(2550,3034):
    try:
        frame = init_video[k]
        stretarr = full_logo_recognition(frame, st)
        swretarr = full_logo_recognition(frame, sw)
        if stretarr[0]:
            init_video[k] = draw_frames(frame,stretarr[1][0],stretarr[1][1],stretarr[1][2], [480,130], 0.3, (0, 255, 0))
        if swretarr[0]:
            init_video[k] = draw_frames(frame,swretarr[1][0],swretarr[1][1],swretarr[1][2], [480,130], 0.3, ( 255,0, 0))
    except:
        print (k)
for k in range(3036,6300):
    try:
        frame = init_video[k]
        stretarr = full_logo_recognition(frame, st)
        swretarr = full_logo_recognition(frame, sw)
        if stretarr[0]:
            init_video[k] = draw_frames(frame,stretarr[1][0],stretarr[1][1],stretarr[1][2], [480,130], 0.3, (0, 255, 0))
        if swretarr[0]:
            init_video[k] = draw_frames(frame,swretarr[1][0],swretarr[1][1],swretarr[1][2], [480,130], 0.3, ( 255,0, 0))
    except:
        print (k)
for k in range(6300,6930):
    try:
        frame = init_video[k]
        swretarr = full_logo_recognition(frame, sw)
        if swretarr[0]:
            init_video[k] = draw_frames(frame,swretarr[1][0],swretarr[1][1],swretarr[1][2], [480,130], 0.3, ( 255,0, 0))
    except:
        print (k)
for k in range(6930,len(init_video) - 1):
    try:
        frame = init_video[k]
        stretarr = full_logo_recognition(frame, st)
        swretarr = full_logo_recognition(frame, sw)
        if stretarr[0]:
            init_video[k] = draw_frames(frame,stretarr[1][0],stretarr[1][1],stretarr[1][2], [480,130], 0.3, (0, 255, 0))
        if swretarr[0]:
            init_video[k] = draw_frames(frame,swretarr[1][0],swretarr[1][1],swretarr[1][2], [480,130], 0.3, ( 255,0, 0))
    except:
        print (k)
with open('output.rgb', "wb") as f:
    for p in init_video:
        output_frame = np.ravel(np.split(p, 3, axis = 2), order = 'C')
        f.write (bytes(output_frame))
f.close()
#output avi for debugging
out = cv2.VideoWriter('video.avi', cv2.VideoWriter_fourcc(*'DIVX'), 30, (480, 270))
for i in range(len(init_video)):
    out.write(init_video[i])
out.release()

In [ ]:
k

In [ ]:
scale = cv2.imread('Dataset/Brand Images/{company}_logo.bmp'.format(company = "starbucks"))
scale_percent = 30 # percent of original size
width = int(scale.shape[1] * scale_percent / 100)
height = int(scale.shape[0] * scale_percent / 100)
dim = (width, height)
resized = cv2.resize(scale, dim, interpolation = cv2.INTER_AREA)
testlogo = resized
for k in range(2300,2500):
    testframe = init_video[k]
    retarr = full_logo_recognition(testframe, testlogo)
    if retarr[0]:
        init_video[k] = draw_frames(testframe,retarr[1][0],retarr[1][1],retarr[1][2], [130,130], 0.3)

In [ ]:
testlogo = cv2.imread('Dataset/Brand Images/{company}_logo.bmp'.format(company = "subway"))
for k in range(6750,6950):
    testframe = init_video[k]
    retarr = full_logo_recognition(testframe, testlogo)
    if retarr[0]:
        init_video[k] = draw_frames(testframe,retarr[1][0],retarr[1][1],retarr[1][2], [480,130], 0.3)
    else:
        print(len(retarr[1][0]))

In [ ]:

data = Image.fromarray(init_video[2367]).convert('RGB')
        #print(data)
        #convert rgb frame to png for writing
data.save('images/box-test.png'.format(1));

In [ ]:
#output avi for debugging
out = cv2.VideoWriter('video.avi', cv2.VideoWriter_fourcc(*'DIVX'), 30, (480, 270))
for i in range(1):
    out.write(init_video[2367])
out.release()

In [ ]:
#creating transform matrix to draw binding box
#     good_points = matches[:10]
    
    
#     src_pts = np.float32([ logopoints[m.queryIdx].pt for m in good_points]).reshape(-1,1,2)
#     dst_pts = np.float32([ framepoints[m.trainIdx].pt for m in good_points]).reshape(-1,1,2)
#     M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
#     matchesMask = mask.ravel().tolist()
#     h,w = logo.shape[:2]
#     pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)

#     dst = cv2.perspectiveTransform(pts,M)
#     dst += (w, 0)  # adding offset

#     draw_params = dict(matchColor = (0,255,0), # draw matches in green color
#                singlePointColor = None,
#                matchesMask = matchesMask, # draw only inliers
#                flags = 2)

#     img3 = cv2.drawMatches(logo,logopoints,frame1,framepoints,good_points, None,**draw_params)

# # Draw bounding box in Red
#     img3 = cv2.polylines(img3, [np.int32(dst)], True, (0,0,255),3, cv2.LINE_AA)

#     cv2.imshow("result", img3)
#cv2.waitKey()
# or another option for display output
#plt.imshow(img3, 'result'), plt.show(

In [ ]:
logo_names = ["subway", "starbucks"]
logos = []
for l in logo_names:
    if(l=="starbucks"):
        scale = cv2.imread('Dataset/Brand Images/{company}_logo.bmp'.format(company = l))
        scale_percent = 30 # percent of original size
        width = int(scale.shape[1] * scale_percent / 100)
        height = int(scale.shape[0] * scale_percent / 100)
        dim = (width, height)
        resized = cv2.resize(scale, dim, interpolation = cv2.INTER_AREA)
        logos.append(resized)
    else:    
        logos.append(plt.imread('Dataset/Brand Images/{company}_logo.bmp'.format(company = l)))
#for all the beginning cut sections
for k in range(len(cuts_begin)):
    loopstart = 0
    if not k == 0:
        loopstart = cuts_end[k-1]
    
    #for all the frames before the cut
    for j in range(loopstart,cuts_begin[k],30):
        
        #format each image
        img = np.flip(final_video[j],axis = 2)
        
        tobreak = False
        for logo in range(len(logos)):
            retarr = full_logo_recognition(img, logos[logo])
            #if(retarr[0]):
            final_video[j] = draw_frames(testframe,retarr[1][0],retarr[1][1],retarr[1][2], [480,130], 0.3)
                
                #replaced_advertisements.append(logo_names[logo])
                #print(logo_names[logo])
                #tobreak = True
                #break
        #if tobreak:
           # break

In [ ]:
f = open("cuts_begin.txt", "w")
f.write("cuts_begin.txt")
f.close()

In [ ]:
#saves data for java to cut audio
ad_times = open("ad_times.txt", "w")
for i in range(len(replaced_advertisements)):
    ad_times.write(replaced_advertisements[i] + " " + str(cuts_begin[i]) + "\n")
ad_times.close()

In [ ]:
#add function to find points and draw the box for each frame in the video
#output video to rgb
#
with open('final_output.rgb', "wb") as f:
    for p in final_video:
        output_frame = np.ravel(np.split(p, 3, axis = 2), order = 'C')
        f.write (bytes(output_frame))
f.close()

In [ ]:
#output avi for debugging
out = cv2.VideoWriter('video.avi', cv2.VideoWriter_fourcc(*'DIVX'), 30, (480, 270))
for i in range(len(final_video)):
    out.write(final_video[i])
out.release()

In [ ]:
#output avi for debugging
out = cv2.VideoWriter('video.avi', cv2.VideoWriter_fourcc(*'DIVX'), 30, (480, 270))
for i in range(len(init_video)):
    out.write(init_video[i])
out.release()